# QEC 101 Lab 7: qLDPC Codes #


One of the most promising classes of QEC codes are so called quantum low density parity check (qLDPC) codes. These codes are quite general and include well known codes like the surface code.  This lab will walk through the basics of classical LDPC codes, the challenges that arise when moving to qLDPC codes, and how to construct valid qLDPC codes with favorable properties.  You will eventually implement techniques from "[Lift-Connected Surface Codes](https://arxiv.org/abs/2401.02911)" connecting what you have leaned to state-of-the-art research.

**Prerequisites:** This lab assumes you have a moderate knowledge of QEC and have completed the core QEC 101 courses (labs 1-4), especially the labs covering [stabilizers](https://github.com/NVIDIA/cuda-q-academic/blob/main/qec101/02_QEC_Stabilizers.ipynb) and [decoders](https://github.com/NVIDIA/cuda-q-academic/blob/main/qec101/04_QEC_Decoders.ipynb). 

The list below outlines what you'll be doing in each section of this lab:
* 7.1 Learn the basics of classical LDPC codes and how to analyze their properties.
* 7.2 Learn why quantum LDPC codes are challenging to construct and how to build hypergraph product (HGP) codes.
* 7.3 Extend the HGP procedure to produce larger qLDPC codes with improved properties.
* 7.4 Compare the quality of the codes you created using the NVIDIA BP+OSD decoder.

Terminology you will use:
* low density parity check, encoding rate, degree
* hypergraph product
* lifted product
* circulants


qLDPC codes have a number of favorable properties that make them promising for deployment within nearer term fault tolerant workflows.

First run the cell below to prepare the necessary libraries.

In [1]:
import sys

try:
    import time
    import cudaq_qec as qec
    import galois
    import cudaq_qec
    import ipywidgets as widgets
    import numpy as np
    from IPython.display import display
    from itertools import combinations
    from scipy.sparse import csr_matrix


except ImportError:
    print("Tools not found, installing. Please restart your kernel after this is done.")
    !{sys.executable} -m pip install --upgrade pip
    !{sys.executable} -m pip install galois
    !{sys.executable} -m pip install cudaq-qec
    !{sys.executable} -m pip install ipywidgets
    print("\nNew libraries have been installed. Please restart your kernel!")

## 7.1 Classical LDPC Codes ##

Robert Gallager first conceived of low density parity check (LDPC) codes in his [1960 MIT dissertation](https://dspace.mit.edu/handle/1721.1/11804) but it was underappreciated at the time and interest resurged in the 90's as other error correction codes rose in prominence. LDPC codes are now used widely in telecommunications and computer memory applications

An LDPC code is a classical parity check error correction code with a sparse parity check matrix $H$. The parity check matrix is often represented by a Tanner graph, which was introduced in lab 4 on decoders. A Tanner graph is drawn with check nodes on the top row and variable nodes on the bottom. The Tanner graph for the Steane code is shown below. 

<img src="../Images/qldpc/steanetanner.png" alt="Drawing" style="width: 700px;"/>

A sparse $H$ means that each variable and check node only connects to a limited number of other nodes.  The **variable node degree** characterizes the maximum number of checks any (q)bit is involved in while the and **check node degree** characterizes the maximum number of (q)bits involved in any given check. Ideally, these two values are as small as possible to maintain low density.

A second important property is the **encoding rate** ($r$).

$$ r = \frac{k}{n+c} $$

Where, $k$ is the number of encoded logical bits, $n$ is the number of data bits, and $c$ is the number of check bits.  A high encoding rate is good and means that many logical bits can be encoded with a lower overhead. However, this competes with other properties like the code distance - i.e. the ability to correctly capture errors.

What $k$ is depends on the number of linearly independent constraints. To determine this, perform Gaussian elimination over GF(2).  GF(2) comes from the world of abstract algebra and corresponds to a field of two elements.  Essentially, this just means integer math governed by mod 2 arithmetic.  The Gaussian elimination result can be used to determine rank($H$) which is related to $k$ by 

$$ k = n - \mathrm{rank(}H\mathrm{}) $$


A final characteristic of a desirable LDPC code is how suited it is for decoding.  Common decoders like belief propagation (BP) can struggle when the Tanner graph has 4-cycles.  These form local loops (see image below) which can make it hard for the decoder to converge to a solution.

<img src="../Images/qldpc/fourcycle.png" alt="Drawing" style="width: 200px;"/>

In most cases, it turns out that LDPC codes are very easy to generate. Random generation of $H$ usually produces a good LDPC code. This also provides flexibility as new codes can be generated as needed depending on the problem at hand.  Randomly generated codes also perform well and produce results close to the Shannon limit, that is the theoretical maximum of information that can pass through a noisy channel.


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px;">
    <h3 style="color: #76b900; margin-top: 0;"> Exercise  1:</h3>
    <p style="font-size: 16px; color: #333;">
Given the three parity check matrices below, write a function to analyze them and determine the check and variable node degrees, the encoding rate, the indices of any four cycles, and if any nodes are unchecked.
    </p>
</div>




In [2]:
H1 = np.array([
    [1,1,0,0,1,0,0,0],
    [1,1,0,0,0,1,0,0],
    [0,0,1,1,0,0,1,0],
    [0,0,1,0,1,0,0,1],
    [0,0,0,1,0,1,1,0]], dtype=int)

H2 = np.array([
    [1,0,1,0,0,1,0],
    [0,1,1,1,0,0,0],
    [1,1,0,0,1,0,0],
    [0,0,0,1,1,1,0]], dtype=int)


H3 = np.array([
    [1,0,0,1,0,1,0,0,0,1, 0, 0, 1, 0, 0, 0],  
    [0,1,0,0,1,0,1,0,0,0, 1, 0, 0, 0, 0, 0],
    [0,0,1,0,0,0,0,1,0,0, 0, 1, 0, 1, 1, 0],  
    [1,0,0,0,1,0,0,0,1,0, 0, 0, 0, 1, 0, 0],  
    [0,0,0,0,0,0,1,0,1,0, 0, 1, 1, 0, 0, 0],  
    [0,0,1,0,0,1,0,0,0,0, 1, 0, 0, 0, 0, 1] 
], dtype=int)


def degrees(H):
    """ 
    function which computes the degrees of a parity check matrix
    
    Args:
    H (np.array): parity check matrix

    Returns:
    (list): list of degrees for each variable bit
    (list): list of degrees for each check bit
    """
    
    #TODO
    return H.sum(axis=0), H.sum(axis=1)    # Sums the 1's vertically and then horizontally

def unchecked_vars(H):
    """ 
    function which identifies any unchecked variable bit
    
    Args:
    H (np.array): parity check matrix

    Returns:
    (list): list of unchecked variable bits
    """
    #TODO
    return np.where(H.sum(0) == 0)[0]

def four_cycles(H):
    """ 
    function which identifies any four-cycles in a parity check matrix
    
    Args:
    H (np.array): parity check matrix

    Returns:
    (list): list of nodes involved in a 4-cycle.
    """
    #TODO
    cycles = []
    m, n = H.shape
    for i, j in combinations(range(n), 2):    # variable‑node pairs
        shared = np.where(H[:, i] & H[:, j])[0]
        if shared.size >= 2:
            for p, q in combinations(shared, 2):
                cycles.append((i, j, p, q))
    return cycles

def encoding_rate(H):
    """ 
    function which computes the encoding rate based on rank of H.
    Note: Must use galois for GF2 field definition to ensure computation is correct
    
    Args:
    H (np.array): parity check matrix

    Returns:
    (float): encoding rate
    """
    GF2 = galois.GF(2)
    Hgf2 = GF2(H)
    n = Hgf2.shape[1]
    rank = np.linalg.matrix_rank(Hgf2)          
    k = n - rank
    return k / n, k


def analyze(H, name): 
    """ 
    Function that organizes and prints results from previous functions
    
    Args:
    H (np.array): parity check matrix
    name (str): name of the parity chexk matrix

    Returns:
    """
    #TODO
    vdeg, cdeg = degrees(H)
    R, k        = encoding_rate(H)
    cycles      = four_cycles(H)
    unchk       = unchecked_vars(H)

    print(f'\n{name}:')
    print('  variable degrees:', vdeg.tolist())
    print('  check    degrees:', cdeg.tolist())
    print(f'  rate = {R:.3f}  (k = {k})')

    if cycles:
        print('  4‑cycles:')
        for i, j, p, q in cycles:
            print(f'    vars ({i},{j})  rows ({p},{q})')
    else:
        print('  no 4‑cycles')

    if unchk.size:
        print('  unchecked variables:', unchk.tolist())
    else:
        print('  all variables are checked')


for H, name in [(H1, 'H1'), (H2, 'H2'), (H3, 'H3')]:
    analyze(H, name)


H1:
  variable degrees: [2, 2, 2, 2, 2, 2, 2, 1]
  check    degrees: [3, 3, 3, 3, 3]
  rate = 0.375  (k = 3)
  4‑cycles:
    vars (0,1)  rows (0,1)
    vars (3,6)  rows (2,4)
  all variables are checked

H2:
  variable degrees: [2, 2, 2, 2, 2, 2, 0]
  check    degrees: [3, 3, 3, 3]
  rate = 0.571  (k = 4)
  no 4‑cycles
  unchecked variables: [6]

H3:
  variable degrees: [2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1]
  check    degrees: [5, 4, 5, 4, 4, 4]
  rate = 0.625  (k = 10)
  no 4‑cycles
  all variables are checked


## 7.2 Quantum LDPC ##

qLDPC codes have many similarities to their classical counterparts, particularly with respect to terms like encoding rate and degree. Unfortunately, a major difference is that valid qLDPC codes with favorable properties cannot be produced by randomly generating parity check matrices. This is because the $Z$ and $X$ parity check matrices ($H_Z$ and $H_X$) must commute ($H_ZH^T_X=0$) for a valid CSS code that can correct both types of errors. 

The probability of randomly producing parity check matrices that commute is vanishingly small, let alone exhibit favorable properties. Cutting edge research focused on qLDPC codes is determined to find clever ways to produce quality parity check matrices that meet these constraints.  

One particularly insightful approach is using [so called hypergraph product codes](https://arxiv.org/pdf/2401.02911). The idea is to take two "good" ( in this case a technical term meaning the codes distance scales as $n$) classical parity check matrices $H_1$ ($m_1\times n_1$) and $H_2$ ($m_2\times n_2$) and combine them in such a way that $H_Z$ and $H_X$ commute (i.e. $H_ZH_X^T=0$) and the resulting codes have a constant encoding rate and a distance that scales proportionally to the square root of the number of data qubits.

The procedure works by defining the final parity check matrix $H$ as a block encoding of $H_Z$ and $H_X$. 


$$
\begin{pmatrix}
0 & H_Z  \\
H_X & 0
\end{pmatrix}
$$

Each quantum parity check matrix is then defined in terms of the the two classical base matrices.

$$ H_X =
\begin{pmatrix}
\mathbf{1_{n_1}} \otimes H_2 & H_1^T \otimes \mathbf{1_{m_2}}
\end{pmatrix}
$$

$$ H_Z =
\begin{pmatrix}
H_1 \otimes \mathbf{1_{n_2}} & \mathbf{1_{m_1}} \otimes H_2^T
\end{pmatrix}
$$

This construction ensures that $H_Z$ and $H_X$ commute. The proof is as follows:


$$ H_ZH_X^T  =
\begin{pmatrix}
H_1 \otimes \mathbf{1_{n_2}} & \mathbf{1_{m_1}} \otimes H_2^T
\end{pmatrix}
\begin{pmatrix}
(\mathbf{1_{n_1}} \otimes H_2)^T \\
(H_1^T \otimes \mathbf{1_{m_2}})^T
\end{pmatrix}
$$

$$
= (H_1 \otimes \mathbf{1_{n_2}})(\mathbf{1_{n_1}} \otimes H_2^T) + (\mathbf{1_{m_1}} \otimes H_2^T)(H_1 \otimes \mathbf{1_{m_2}})
$$

$$
= H_1 \otimes H_2^T + H_1 \otimes H_2^T
$$

$$
= 2(H_1 \otimes H_2^T) = 0
$$

It may not be clear at first why the final term equals zero.  Recall that all operations with parity check matrices occur mod 2.  So, taking any binary matrix and multiplying it by 2, will make every entry 0 or 2 = 0 mod 2. 

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px;">
    <h3 style="color: #76b900; margin-top: 0;"> Exercise  2:</h3>
    <p style="font-size: 16px; color: #333;">
Construct a hypergraph product code using a pair of three-qubit repetition code base matrices.  That is, begin with:

$$H_1 =H_2 =\begin{pmatrix}
1&1&0\\
0&1&1
\end{pmatrix}
$$

Build the parity check matrices for $H_Z$ and  $H_X$ and confim they commute. Note: the `galois` package is used to define the matrices over a Galois field (GF2), which ensures modular arithmetic is baked in to your computations.  All operations can be performed just like you would with `numpy`.
    </p>
</div>





In [3]:
H = np.array([[1,1,0],
              [0,1,1]])   #Using H as H1  = H2

def HGP(H):
    """ 
    Function which takes classical base parity check matricies and performs hypergraph product construction
    
    Args:
    H (np.array): Base parity check matrix

    Returns:
    Hz (np.array): Hz matrix from HGP construction
    Hx (np.array): Hx matrix from HGP construction
    """
    #TODO Start
    rows, cols = H.shape

    I_rows = np.eye(rows, dtype=int)    
    I_cols = np.eye(cols, dtype=int)

    #TODO END
   
    # Constructs a Galois field and updates you matricies at Galois field.
    GF2 = galois.GF(2) 
    
    H = GF2(H)                               
    I_rows = GF2(I_rows)
    I_cols = GF2(I_cols)

    #TODO start
    print("First term in Hx")
    Hx_a = np.kron(I_cols, H)
    print(Hx_a)
    
    print("\n Second term in Hx")
    Hx_b = np.kron(H.T,I_rows)
    print(Hx_b)
    
    print("\n Full Hx")
    Hx = np.concatenate((Hx_a, Hx_b), axis=1)
    print(Hx)
    
    print("\nFirst term in Hz")
    Hz_a = np.kron(H, I_cols)
    print(Hz_a)
    
    print("\n Second term in Hz")
    Hz_b = np.kron(I_rows, H.T)
    print(Hz_b)
    
    print("\n Full Hz")
    Hz = np.concatenate((Hz_a, Hz_b), axis=1)
    print(Hz)
    
    print("\n Hz times HxT")
    print(Hz @ Hx.T)

    return Hz, Hx
    
HGP(H)
#TODO END

First term in Hx
[[1 1 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 1 1]]

 Second term in Hx
[[1 0 0 0]
 [0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 1 0]
 [0 0 0 1]]

 Full Hx
[[1 1 0 0 0 0 0 0 0 1 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 1 0 0 0 0 1 0 1 0]
 [0 0 0 0 1 1 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 1]]

First term in Hz
[[1 0 0 1 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0]
 [0 0 1 0 0 1 0 0 0]
 [0 0 0 1 0 0 1 0 0]
 [0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 1 0 0 1]]

 Second term in Hz
[[1 0 0 0]
 [1 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 1]
 [0 0 0 1]]

 Full Hz
[[1 0 0 1 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 1 0 0 0 0 1 1 0 0]
 [0 0 1 0 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 0 1 1]
 [0 0 0 0 0 1 0 0 1 0 0 0 1]]

 Hz times HxT
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


(GF([[1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
     [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0],
     [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
     [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1],
     [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]], order=2),
 GF([[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
     [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
     [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0],
     [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],
     [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]], order=2))

It turns out there is a nice visual interpretation of the hypergraph product code you just generated if the Tanner graphs form a multiplication table of sorts.  Each node of the product tanner graph that is the product of a check qubit with a check qubit or a data qubit with a data qubit produces a data qubit. If the top Tanner graph is a circle (data qubit) and the left Tanner graph a square (check qubit), the result is an $X$ stabilizer check.  Likewise, if the top Tanner graph is a square and the left Tanner graph a circle, the result is a $Z$ parity check, producing the tanner graph below. Does it look familiar?

<img src="../Images/qldpc/hgp.png" alt="Drawing" style="width: 700px;"/>


Remarkably, it turns out the product of two size $l$ classical repetition codes is a [[ $(l+1)^2 + l^2$, $1$, $l+1)$]] surface code! This is a great example demonstrating how two very simple classical codes can construct a more sophisticated quantum code which obeys the required commutativity constraints.

## 7.3 Generalizing HGP - Lifted Product Codes

It is possible to build upon the HGP method in a more general manner where products are taken between two parity check matrices that have non-integer entries. Such an approach is called a **lifted product (LP)** as the elements of the parity check matrix are "lifted" to higher order elements.  LP codes can often retain the degree of checks and provide higher distance codes with a smaller qubit overhead. 

A LP construction still needs to ensure that $H_ZH_X^T=0$ holds as parity check matrices are modified to have non-integer elements. One way to ensure this is to replace parity check matrix elements with a commutative matrix ring, that is, a set of mathematical objects with properties that ensure multiplication of any elements commute, ensuring $H_ZH_X^T=0$ remains true (see the second term in the original proof of commutativity a few cells above). One example is $L \times L$ **circulant** matrices which are defined as:

$$ C = \sum_{i=0}^{L-1} c_iP^{(i)} $$

Where $P^{(i)}$ are cyclic permutation matrices that shift columns of the identity matrix by $i$ spaces to the right and where $c_i$ can be either 0 or 1. The notation $B_L(P^{(i)})$ indicates the binary representation of matrix size $L$. 


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px;">
    <h3 style="color: #76b900; margin-top: 0;"> Exercise  3:</h3>
    <p style="font-size: 16px; color: #333;">
Build the following binary matrix representations:

* $B_4(P^{(2)})$
* $B_5(P^{(1)}+P^{(2)})$
* $B_3\begin{pmatrix}
I&0\\
0&P^{(2)}
\end{pmatrix}$
    </p>
</div>



In [4]:
#TODO 

arr1 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])


arr2 = np.array([
    [0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1],
    [1, 0, 0, 0, 1],
    [1, 1, 0, 0, 0]
])


arr3 = np.array([
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0]
])


A LP code is built by taking a HGP of two parity check matrices where each 1 is, in this case, a circulant matrix $C$. A LP code of size $L$ will increase the number of qubits and checks by a factor of L. This is because each entry in the parity check matrix is "lifted" and replaced by a set of $L$ checks and qubits. 

The same HGP equations from the previous section hold, but instead, the parity check matrices are denoted with a tilde to note that their elements are circulants where each entry is otherwise a 1. That is to say, $H=LP(\tilde{H}_1,\tilde{H}_2) = B_L(\tilde{H})$

The figure below, based on figure 2 of [Lift-Connected Surface Codes](https://arxiv.org/pdf/2401.02911), is helpful for understanding what the LP construction is doing. The overall procedure is similar.  First, the base matrices are selected and are used in the same HGP procedure you did previously to form $\tilde{H}_Z$ and $\tilde{H}_X$.  Then, each 1 in each parity check matrix is replaced with a circulant $C$. This simple swap takes select connections from the original Tanner graph, and lifts it to be replaced with a set of check and data qubits. 

<img src="../Images/qldpc/lifted.png" alt="Drawing" style="width: 1000px;"/>

If $C$ is simply the identity matrix $I$, the resulting LP codes is the result of the HGP code duplicated trivially $L$ times.  Adding permutations to $C$ such as $I + P^{(1)}$ adds non-trivial checks between the HGP code copies. Notice the figure below (adapted figure 3 from [Lift-Connected Surface Codes](https://arxiv.org/pdf/2401.02911)) begins with the HGP that you performed earlier.  Then, The LP construction creates four copies of the surface code and interconnects them with parity checks.

<img src="../Images/qldpc/LP_SC.png" alt="Drawing" style="width: 1100px;"/>

It turns out that the LP surface code is a [[52,4,4]] code whereas four copies of the surface code would be [[52,4,3]]. This means that the encoding rate is the same, but the code distance improves thanks to the LP procedure.  These sorts of clever constructions are driving qLDPC code research to continually improve code properties while maintaining the commutativity properties. 

Generally speaking, a LP surface code is parameterized with $l$ and $L$, where $l$ is the size of the base repetition code and $L$ is the number of copies produced by the lift procedure.




<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px;">
    <h3 style="color: #76b900; margin-top: 0;"> Exercise  4:</h3>
    <p style="font-size: 16px; color: #333;">
Build the [[52,4,4]] ( $l =2$ and $L =4$) LP surface code by performing the following steps. First, use the base matrix below which can be conveniently split into $H_{copy}$, which produces trivial copies of the surface code and ($H_{int}$), which interacts these surface code copies.

$
\begin{pmatrix}
I & I + P^{(1)} & 0 & \cdots & \cdots &0 \\
0 & I & I + P^{(1)} & \cdots & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots &\vdots \\
0 & \cdots & 0 & I & I + P^{(1)}  & 0\\
0 & \cdots & \cdots & 0 & I &I+ P^{(1)}
\end{pmatrix}=
\begin{pmatrix}
I & I & 0 & \cdots & \cdots &0 \\
0 & I & I & \cdots & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots &\vdots \\
0 & \cdots & 0 & I & I  & 0\\
0 & \cdots & \cdots & 0 & I &I
\end{pmatrix}
+
\begin{pmatrix}
0 & P^{(1)} & 0 & \cdots & \cdots &0 \\
0 & 0 & P^{(1)} & \cdots & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots &\vdots \\
0 & \cdots & 0 & 0 & P^{(1)}  & 0\\
0 & \cdots & \cdots & 0 & 0 & P^{(1)}
\end{pmatrix}
$

To produce the [[52,4,4]] code, we can perform the HGP procedure twice using $
H =\begin{pmatrix}
1 & 1 & 0 \\
0 & 1 & 1  \\
\end{pmatrix} 
$ and $H =\begin{pmatrix}
0 & 1 & 0 \\
0 & 0 & 1  \\
\end{pmatrix} 
$. Lifting the first with $B_4(I)$ and the second with $B_4(P^{(1)})$ and summing the results will produce the parity check matrices for the [[52,4,4]] code.

Modify your HGP function to lift `Hx_a` and `Hz_a` with an arbitrary $B$ and `Hx_b` and `Hz_b` with the transpose of $B$.  

Then build the [[52,4,4]] code by lifting H_copy and H_int (defined below) with $B_4(I)$ and $B_4(P^{(1)})$ , respectively. Confirm that Hz and Hx of the final result commute.
    </p>
</div>



In [5]:
H_copy = np.array([[1,1,0],
                   [0,1,1]])   

B_I_4 = np.array([[1,0,0,0],
                   [0,1,0,0],
                   [0,0,1,0],
                   [0,0,0,1]])   

H_int = np.array([[0,1,0],
                   [0,0,1]])  

B_P1_4 = np.array([[0,1,0,0],
                    [0,0,1,0],
                    [0,0,0,1],
                    [1,0,0,0]])   



def LP(H, B):
    """ 
    Function which perfoms lifted product construction of base matrices H with lift matrix B
    
    Args:
    H (np.array): Base parity check matrix
    B (np.array): Binary representation of lift matrix

    Returns:
    Hz (np.array): Hz matrix from HGP construction
    Hx (np.array): Hx matrix from HGP construction
    """

    rows, cols = H.shape

    I_rows = np.eye(rows, dtype=int)
    I_cols = np.eye(cols, dtype=int)
   
    GF2 = galois.GF(2) # allows mod 2 math.
    
    H = GF2(H)
    I_rows = GF2(I_rows, dtype=int)
    I_cols = GF2(I_cols, dtype=int)
    B = GF2(B)

    print("First term in Hx")
    Hx_a = np.kron(I_cols, H)
    print(Hx_a)

    print("First term in Hx lifted")
    Hx_a = np.kron(Hx_a, B)
    print(Hx_a)
    
    print("\n Second term in Hx")
    Hx_b = np.kron(H.T,I_rows)
    print(Hx_b)

    print("Second term in Hx lifted")
    Hx_b = np.kron(Hx_b, B.T)
    print(Hx_b)
    
    print("\n Full Lifted Hx")
    Hx = np.concatenate((Hx_a, Hx_b), axis=1)
    print(Hx)
    
    print("\nFirst term in Hz")
    Hz_a = np.kron(H, I_cols)
    print(Hz_a)

    print("First term in Hz lifted")
    Hz_a = np.kron(Hz_a, B)
    print(Hz_a)
    
    print("\n Second term in Hz")
    Hz_b = np.kron(I_rows, H.T)
    print(Hz_b)

    print("Second term in Hz lifted")
    Hz_b = np.kron(Hz_b, B.T)
    print(Hz_b)
    
    print("\n Full Hz")
    Hz = np.concatenate((Hz_a, Hz_b), axis=1)
    print(Hz)
    
    print("\n Hz times HxT")
    print(Hz @ Hx.T)

    return Hz, Hx

Hz_lifted_copy, Hx_lifted_copy = LP(H_copy, B_I_4)


First term in Hx
[[1 1 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 1 1]]
First term in Hx lifted
[[1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0

In [6]:
Hz_lifted_int, Hx_lifted_int = LP(H_int, B_P1_4)

First term in Hx
[[0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1]]
First term in Hx lifted
[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0

In [7]:
Hz_lifted_total = Hz_lifted_copy + Hz_lifted_int
Hx_lifted_total = Hx_lifted_copy + Hx_lifted_int

Now, analyze the Hz and Hx parity check matrices to 1) make sure they commute and 2) confirm the degrees are as expected. Each stabilizer should act on maximum 6 qubits and each qubit should be involved in no more than 6 checks (summing Z and X checks as the full parity check matrix would be a concatenation of both $H_x$ and $H_z$.). 

In [8]:
#Confirm Hz and Hx still commute
print("\n Hz times HxT")
print(Hz_lifted_total @ Hx_lifted_total.T)

#Confirm [[52,4,4]] code was created
print("\n Number of logical qubits encoded (data qubits minus checks)")
print(Hz_lifted_total.shape[1]-2*Hz_lifted_total.shape[0])

#Confirm degree of code is correct (should be 6 and 6)
analyze(Hz_lifted_total.view(np.ndarray), 'hz lifted')
analyze(Hx_lifted_total.view(np.ndarray), 'hx lifted')


 Hz times HxT
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 

## 7.4 Decoding with CUDA-Q QEC

💻 Just a heads-up: This notebook is designed to be run on an environment with a GPU. If you don't have access to a GPU, feel free to read through the cells and explore the content without executing them. Enjoy learning! ⭐

qLDPC codes are well suited for decoding with CUDA-Q's accelerated [BP+OSD decoder](https://nvidia.github.io/cudaqx/components/qec/introduction.html#pre-built-qec-decoders) found in the [CUDA-Q QEC library](https://nvidia.github.io/cudaqx/components/qec/introduction.html). If you want to learn more about BP+OSD decoding, complete lab 4 on decoding.

As we have not discussed logical observables for these codes, the code below will randomly generate an error vector with a 5% chance of an error on each qubit (Only assuming bit flip errors for now). The decoder will produce a logical error if the decoder cannot identify all of the errors given the syndrome.  However, note that each of these errors might not produce a logical flip in practice, so this considers a worst case scenario.  

In the cell below, run the decoder using the $H_z$ matrix you produced for the [[52,4,4]] LP surface code. Carefully read the code and see if you can spot where errors and syndromes are generated, where decoder options are specified, and how the decoder is called. Run the code and note what the logical error rate is.

In [9]:
# Define parameters
n = Hz_lifted_total.shape[1]  # number of physical qubits
k = n - Hz_lifted_total.shape[0]  # number of logical qubits
num_shots = 10000

# Define error rates (can be uniform or varied per qubit)
error_rate_vec = np.ones(n) * 0.05  # error rate for each qubit

# Create decoder
nv_dec_args = {
    "max_iterations": 50,
    "error_rate_vec": error_rate_vec,
    "use_sparsity": True,
    "use_osd": True,
    "osd_order": 0,
    "osd_method": 0,
    "bp_batch_size": 1000
}

decoder = qec.get_decoder("nv-qldpc-decoder", Hz_lifted_total, **nv_dec_args)

# Generate random errors and decode
decoding_time = 0
bp_converged_flags = []
num_logical_errors = 0

# Generate batch of errors
errors = np.random.binomial(1, error_rate_vec, (num_shots, n))
syndromes = (Hz_lifted_total.view(np.ndarray) @ errors.T % 2).T

# Batched decoding
t0 = time.time()
results = decoder.decode_batch(syndromes.tolist())
t1 = time.time()
decoding_time = t1 - t0

# Process results
for i, r in enumerate(results):
    bp_converged_flags.append(r.converged)
    decoded_error = np.array(r.result, dtype=np.uint8)
    
    # Check if error was corrected
    if not np.array_equal(decoded_error, errors[i]):
        num_logical_errors += 1

# Print statistics
print(f"{num_logical_errors} logical errors in {num_shots} shots")
print(f"Number of shots that converged with BP: {sum(bp_converged_flags)}")
print(f"Average decoding time: {1e3 * decoding_time / num_shots:.2f} ms per shot")

# Optional: Single shot example
single_syndrome = syndromes[0]
bp_converged, decoded_result, *_ = decoder.decode(single_syndrome.tolist())


1694 logical errors in 10000 shots
Number of shots that converged with BP: 7966
Average decoding time: 0.01 ms per shot


Now, run the same code but use the `Hz_lifted_copy` parity check matrix. This code is simply four non-interacting copies of the surface code and hence a [[52,4,3]] code.  How does the logical error rate compare?  Can you see the benefit of the LP surface code as it adds one to the distance and outperforms copies of the surface code significantly.

In [11]:
# Define parameters
n = Hz_lifted_copy.shape[1]  # number of physical qubits
k = n - Hz_lifted_copy.shape[0]  # number of logical qubits
num_shots = 10000

# Define error rates (can be uniform or varied per qubit)
error_rate_vec = np.ones(n) * 0.05  # error rate for each qubit

# Create decoder
nv_dec_args = {
    "max_iterations": 50,
    "error_rate_vec": error_rate_vec,
    "use_sparsity": True,
    "use_osd": True,
    "osd_order": 0,
    "osd_method": 0,
    "bp_batch_size": 1000
}

decoder = qec.get_decoder("nv-qldpc-decoder", Hz_lifted_copy, **nv_dec_args)

# Generate random errors and decode
decoding_time = 0
bp_converged_flags = []
num_logical_errors = 0

# Generate batch of errors
errors = np.random.binomial(1, error_rate_vec, (num_shots, n))
syndromes = (Hz_lifted_copy.view(np.ndarray) @ errors.T % 2).T

# Batched decoding
t0 = time.time()
results = decoder.decode_batch(syndromes.tolist())
t1 = time.time()
decoding_time = t1 - t0

# Process results
for i, r in enumerate(results):
    bp_converged_flags.append(r.converged)
    decoded_error = np.array(r.result, dtype=np.uint8)
    
    # Check if error was corrected
    if not np.array_equal(decoded_error, errors[i]):
        num_logical_errors += 1

# Print statistics
print(f"{num_logical_errors} logical errors in {num_shots} shots")
print(f"Number of shots that converged with BP: {sum(bp_converged_flags)}")
print(f"Average decoding time: {1e3 * decoding_time / num_shots:.2f} ms per shot")

# Optional: Single shot example
single_syndrome = syndromes[0]
bp_converged, decoded_result, *_ = decoder.decode(single_syndrome.tolist())

2952 logical errors in 10000 shots
Number of shots that converged with BP: 8396
Average decoding time: 0.01 ms per shot


Each choice of $l$ and $L$ will produce a different LP surface code.  It varies case by case if the LP approach is better than copies of the surface code. Examine the table below from [Lift-Connected Surface Codes](https://arxiv.org/pdf/2401.02911) where the code parameters were obtained via numerical simulation. Note, how the entries highlighted in green are cases where the LP construction results (top entry) in a code with the same overhead but a higher code distance compared to surface code copies (bottom entry).


<img src="../Images/qldpc/LP_table.png" alt="Drawing" style="width: 1100px;"/>

## Summary

You now have a foundational understanding of qLDPC codes and how they differ from their classical counterparts.  qLDPC codes are quite promising and will continue to be an active field of research.  The methods covered in this work are just a sample of the different ways to construct qLDPC code parity check matrices yet lay the groundwork for you to understand other state of the art techniques like [bivariate bicycle codes](https://arxiv.org/abs/2308.07915).